In [19]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn.model_selection import train_test_split

In [2]:
df=pd.read_csv("/kaggle/input/2010-2019-fall-marathons/Results.csv",nrows=10000)

In [3]:
df

,Race,Year,Name,Gender,Age,Finish,Age Bracket
0,Columbus Marathon,2015,Becky Williams,F,29,13645,Under 35
1,Columbus Marathon,2015,Jessica Sheahan,F,27,13648,Under 35
2,Columbus Marathon,2015,Kelsey Stief,F,27,13650,Under 35
3,Columbus Marathon,2015,Mehgan Cline,F,26,13650,Under 35
4,Columbus Marathon,2015,Stefanie Driesenga,F,28,13652,Under 35
...,...,...,...,...,...,...,...
9995,Empire State Marathon,2013,Gin Nelson,F,52,17001,50-54
9996,Empire State Marathon,2013,Norman Moore,M,74,17020,70-74
9997,Empire State Marathon,2013,Robert Wood,M,56,17025,55-59
9998,Empire State Marathon,2013,John Mulhall,M,44,17052,40-44


In [4]:
# label_encoder=LabelEncoder()
# df["Racel"]=label_encoder.fit_transform(df["Race"])
# df["Namel"]=label_encoder.fit_transform(df["Name"])
# df["Genderl"]=label_encoder.fit_transform(df["Gender"])
# df["Age Bracketl"]=label_encoder.fit_transform(df["Age Bracket"])

In [5]:
one_hot=OneHotEncoder(sparse_output=False)
one=one_hot.fit_transform(df[["Race","Name","Gender"]])
one=pd.DataFrame(one,columns=one_hot.get_feature_names_out())
one_hot_age=OneHotEncoder(sparse_output=False)
one_age=one_hot_age.fit_transform(df[["Age Bracket"]])
one_age=pd.DataFrame(one_age,columns=one_hot_age.get_feature_names_out())

In [12]:
one_age.nunique().sum()

22

In [6]:
concaty=pd.concat([df,one],axis=1)
concaty=concaty.drop(columns=["Race","Name","Gender","Age Bracket"])

In [13]:
concaty.shape

(10000, 9956)

In [24]:
X_train,X_test,Y_train,Y_test=train_test_split(concaty,one_age,test_size=0.2,random_state=42)

# # Deep learning

In [20]:
from keras.layers import Dense,BatchNormalization,TimeDistributed,Input,Dropout
from keras.models import Model
from keras.initializers import HeNormal
from keras.regularizers import l2
from keras.callbacks import EarlyStopping

In [21]:
early_stopping=EarlyStopping(
monitor='val_loss',
patience=5,
verbose=1
)

In [33]:
inputs=Input(shape=(9956,))
D=Dense(32,activation="relu",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(inputs)
D=Dropout(0.5)(D)
D=Dense(32,activation="relu",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(D)
B=BatchNormalization()(D)
outputs=Dense(11,activation='softmax')(B)
model=Model(inputs,outputs)
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=["accuracy"])

In [34]:
model.fit(X_train,Y_train,epochs=50,batch_size=32,validation_data=(X_test,Y_test),callbacks=[early_stopping])

Epoch 1/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.1719 - loss: 3.1378 - val_accuracy: 0.4005 - val_loss: 2.0849
Epoch 2/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.3903 - loss: 2.0434 - val_accuracy: 0.4005 - val_loss: 1.8603
Epoch 3/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.3921 - loss: 1.8692 - val_accuracy: 0.4005 - val_loss: 1.7921
Epoch 4/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.4035 - loss: 1.7952 - val_accuracy: 0.4005 - val_loss: 1.7728
Epoch 5/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.3962 - loss: 1.8054 - val_accuracy: 0.4005 - val_loss: 1.7576
Epoch 6/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.3949 - loss: 1.7896 - val_accuracy: 0.4005 - val_loss: 1.7531
Epoch 7/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.4028 - loss: 1.7637 - val_accuracy: 0.4005 - val_loss: 1.7532
Epoch 8/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.3989 - loss: 1.7713 - val_accuracy: 0.

In [35]:
loss,accuracy=model.evaluate(X_test,Y_test)

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3874 - loss: 1.7593
